In [2]:
import re
from re import search
from nltk.corpus import stopwords
import string
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
# emotion_analyzer = create_analyzer(task="emotion", lang="es")
# hate_analyzer = create_analyzer(task="hate_speech", lang="es")
from unidecode import unidecode

ModuleNotFoundError: No module named 'pysentimiento'

In [2]:
df = pd.read_excel('data/export1.xlsx')
df['date'] = pd.to_datetime(df['fecha'])
df['hour'] = df['date'].dt.hour
df['date2'] = df['date'].dt.date
df = df[['texto', 'date2', 'hour']]
df.columns = ['tweet', 'date', 'hour']
name = 'Mall_Plaza'
df_tweets = df
df_tweets

,tweet,date,hour
0,RT @CarlosVerareal: Quieren quebrar a todos lo...,2020-12-25,23
1,RT @tanquedecasma: Normalicemos llamar Golpe d...,2020-12-25,23
2,RT @PaoladrateleSUR: A sólo dos semanas del ci...,2020-12-25,23
3,RT @HinterlacesNet: ✝️🛐 José Vicente Rangel: U...,2020-12-25,23
4,RT @pescobio: Lo país lo pinta como una aventu...,2020-12-25,23
...,...,...,...
24995,RT @BCV_ORG_VE: Atención🔵|| El tipo de cambio ...,2020-12-23,21
24996,RT @BCV_ORG_VE: Atención🔵|| El tipo de cambio ...,2020-12-23,21
24997,RT @BCV_ORG_VE: Atención🔵|| El tipo de cambio ...,2020-12-23,21
24998,🤩 ¡CIERRE DE LOCOS EN SAN SIRO! \n\n⚽ ¡THEO HE...,2020-12-23,21


In [3]:
# Cargar stopwords en español
stopwords_es = stopwords.words('spanish')

"""
Excluir menciones, emails, URLs y simbolos
"""
def clean_tweet(tweet):
    # Convertir a minusculas
    tweet = tweet.lower()

    # Excluir menciones o emails
    tweet = re.sub(r'\w*@(\w+\.*\w+\.*\w+)',' ', tweet)

    # Excluir simbolos
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Excluir URLs
    tweet = re.sub(r'(?:www\.|https?)[^\s]+', ' ', tweet, flags=re.MULTILINE)

    # Borrar espacios
    tweet = tweet.strip()

    # Considerar solo valores alfa numericos
    tweet_alfa = re.compile("^(?![0-9]*$)[a-zA-Z0-9]+$")

    # Eliminar stopwords y palabras con longitud <= 2
    tokens = tweet.split()
    text = [token for token in tokens if token not in stopwords_es and len(token)>2 and tweet_alfa.match(token)]
    return ' '.join(text)

# Aplicar filtro a tweets
df_tweets['tweet_cleaned'] = df_tweets['tweet'].apply(lambda x: clean_tweet(str(x)))
df_tweets['tweet_cleaned'] = df_tweets['tweet_cleaned'].apply(lambda x: preprocess_tweet(x))

In [4]:
#Identificación de subtemas dentro de los temas
df_topic = pd.read_excel("risk.xlsx", sheet_name="versión 3")
df_filtro = df_topic.loc[0,:].reset_index()
df_filtro.columns = ['tema', 'subtema']
tema = df_filtro.tema.to_list()
for i, t in enumerate(tema):
    if 'Unnamed:' in t:
        tema[i] = tema[i-1]
df_filtro.tema = tema
df_filtro = df_filtro.dropna()

In [5]:
#Identificación de palabras clave por subtema
df_topic = pd.read_excel("risk.xlsx", sheet_name="versión 3", header=[1])
for i in range(len(df_topic.columns)):
    subtema = (df_topic.columns[i])
    words = df_topic[df_topic.columns[i]].dropna().to_list()
    words = [w.lower() for w in words]
    pattern = '|'.join(words)
    topic = df_filtro[df_filtro['subtema'] == str(subtema)].tema.to_list()[0]
    column_name = 'isin_' + str(df_topic.columns[i]) + '_' + str(topic)
    df_tweets[column_name] = df_tweets['tweet_cleaned'].str.contains(pattern)

In [6]:
#Identificar los temas mencionados en los tweets de acuerdo a palabras clave
columns = df_tweets.columns
columns_subtopic = [lis for lis in columns if 'isin' in lis]
is_relevant = []
for i in range(len(df_tweets)):
    is_relevant.append(any(df_tweets[columns_subtopic].loc[i,:].to_list()))
df_tweets['relevant'] = is_relevant

In [ ]:
#Identificación de sentimiento
df_tweets['sentiment'] = df_tweets['tweet_cleaned'].apply(lambda x: sentiment_analyzer.predict(x).output)
# df_tweets['hate'] = df_tweets['tweet_cleaned'].apply(lambda x: hate_analyzer.predict(x).output)

In [ ]:
item = 'isin_Covid_Salud'
sentiment_options = ['POS', 'NEG', 'NEU']
df_aux = df_tweets[df_tweets[item] == True][['date', 'hour', 'sentiment']]
df_aux['date'] = pd.to_datetime(df_aux['date'], format='%Y-%m-%d')
df_aux['hour'] = pd.to_datetime(df_aux['hour'], format='%H')
df_aux['time'] = pd.to_datetime(df_aux.date.astype(str) + ' ' + df_aux.hour.astype(str)).dt.strftime("%Y-%m-%d %H:%M:%S")
pd.get_dummies(df_aux, columns = ['sentiment']).groupby('time').sum()